In [1]:
import pandas as pd
import numpy as np
import time
import cantera as ct
import matplotlib.pyplot as plt
from scipy.stats import qmc

# Mechanism

In [2]:
mechanism_path= "C:\\Users\\barki\\Desktop\\master thesis\\papers\\mechanism\\ScienceDirect_files_18Dec2025_16-58-12.343\\C2H4_2021.yaml"
uncertainty_factors = {"C2H4 + O <=> CH3 + HCO" : [0.135, 0.142],
                       "C2H4 + O <=> CH2CO + H2" : [0.135, 0.142],
                       "C2H4 + O <=> CH2CHO + H" : [0.135, 0.142],
                       "C2H4 + OH <=> C2H3 + H2O" : [0.301, 0.318],
                       "C2H4 + OH <=> CH2CH2OH" : [0.143, 0.149],
                       "C2H4 + H (+M) <=> C2H5 (+M)" : [0.22, 0.22],
                       "C2H4 + H <=> C2H3 + H2" : [0.847, 0.898],
                       "C2H5 + O2 <=> C2H4 + HO2" : [0.231, 0.241],
                       "C2H5 + O2 <=> C2H5O2" : [0.491, 0.511]
                      }
for reaction, factor in uncertainty_factors.items():
    uncertainty_factors[reaction] = np.mean(factor)
    
# 278,279,280, f = 0.135 - 0.142
# 271, f = 0.301 - 0.318
# 275, f = 0.143 - 0.149
# 269 , f = 0.22
# 270, f = 0.847 - 0.898
# 316, f = 0.231 - 0.241
# 317, f = 0.491 - 0.511
# gas = ct.Solution('C2H4_2021.yaml')
# rxn_ids = [278,279,280,271,275,269,270,316,317]
# for rxn_id in rxn_ids:
#     base_rxn = gas.reaction(rxn_id)
#     print(f"Reaction {rxn_id}: {base_rxn.equation}")

# IDT

In [3]:
def ignition_delay(states, species):
    """
    This function computes the ignition delay from the occurence of the
    peak in species' concentration.
    """
    i_ign = states(species).Y.argmax()
    return states.t[i_ign]


def calculate_ignition_delay_const_pressure_batch_reactor(T5_list, P5_list, reactants, plot=False):
    # batch reactor adiabatic constant pressure
    ignition_delay_times = []
    for T5, P5 in zip(T5_list, P5_list):
        gas = ct.Solution("CH4_1.0.yaml")
        
        # Define the reactor temperature and pressure
        reactor_temperature = T5  # Kelvin
        reactor_pressure_atm = P5  # atm
        reactor_pressure = reactor_pressure_atm * 101325  # Pascals

        gas.TPX = reactor_temperature, reactor_pressure, reactants
        #print("Mole fractions:", gas.X)

        reactor = ct.IdealGasConstPressureReactor(gas, energy='on')
        reactor_network = ct.ReactorNet([reactor])

        time_history = ct.SolutionArray(gas, extra="t")

        reference_species = "OH"

        t0 = time.time()

        estimated_ignition_delay_time = 0.01
        t = 0

        counter = 1
        while t < estimated_ignition_delay_time:
            t = reactor_network.step()
            if not counter % 10:
                time_history.append(reactor.thermo.state, t=t)
            counter += 1

        tau = ignition_delay(time_history, reference_species)
        
        ignition_delay_times.append(tau)

        t1 = time.time()

        print(f"T5 = {T5} K, P5 = {P5} atm, Computed Ignition Delay: {tau:.3e} seconds. Took {t1-t0:3.2f}s to compute")
        
    if plot:
        scaled_inverse_T5 = [1000.0 / T for T in T5_list]

        plt.figure(figsize=(8, 6))
        plt.scatter(scaled_inverse_T5, ignition_delay_times, label='Cantera', color='blue')
        plt.xlabel('1000/T5 (1/K)')
        plt.ylabel('Ignition Delay Time (s)')
        plt.yscale('log')
        plt.legend()
        plt.title('Ignition Delay Times Comparison')
        plt.grid(True)
        plt.show()
        
    return ignition_delay_times

# sample Mechanism params


In [4]:
dim = uncertainty_factors.__len__()
sampler = qmc.LatinHypercube(dim)
sample = sampler.random(n=1000)
#print(sample)
print(qmc.discrepancy(sample))
# l_bounds = np.ones(dim)
# u_bounds = np.ones(dim)*5
l_bounds = [1,2,3,4,5,56,7,8,9]
u_bounds = [10,49,38,27,16,100,14,13,12]
qmc.scale(sample, l_bounds, u_bounds)
print(qmc.discrepancy(sample))

# uncertainty_intervals = uncertainty_factors
# for key, value in enumerate(uncertainty_intervals.items()):

0.0024787714640992498
0.0024787714640992498


In [5]:
def calc_uncertainty_interval(self):
        # Here f is not temperature dependent
        # TODO: implment for f(T) 
        A_min = self.nominal_arrhenius_params[0] / np.pow(10, self.f)
        A_max = self.nominal_arrhenius_params[0] * np.pow(10, self.f)
        self.A_interval = (A_min, A_max)
gas = ct.Solution('C2H4_2021.yaml')

#rxn_id = 0
#base_rxn = gas.reaction(rxn_id)
# print(f"Reaction {rxn_id}: {base_rxn.equation}")
# base_A = base_rxn.rate.pre_exponential_factor
# base_n = base_rxn.rate.temperature_exponent  
# base_Ea = base_rxn.rate.activation_energy
#base_Ea_kelvin = base_Ea / ct.gas_constant  # Convert from J/mol to K

#print(f"Reaction {rxn_id} original parameters: A={base_A}, n={base_n}, Ea={base_Ea_kelvin}")
print(gas.reactions()[0].equation)
print("2 HO2 <=> H2O2 + O2" in gas.reactions()) 

for i, rxn in enumerate(gas.reactions()):
    #print(f"Reaction {i}: {rxn.equation} | Type: {rxn.reaction_type}")
    
    if rxn.reaction_type == 'Arrhenius' or rxn.reaction_type == 'three-body-Arrhenius':
        A = rxn.rate.pre_exponential_factor
        # b = rxn.rate.temperature_exponent
        # Ea = rxn.rate.activation_energy # J/kmol
    elif rxn.reaction_type == 'falloff-Troe' or rxn.reaction_type == 'falloff-Lindemann':
        A_low = rxn.rate.low_rate.pre_exponential_factor
        # b_low = rxn.rate.low_rate.temperature_exponent
        # Ea_low = rxn.rate.low_rate.activation_energy
        A_high = rxn.rate.high_rate.pre_exponential_factor
        # b_high = rxn.rate.high_rate.temperature_exponent
        # Ea_high = rxn.rate.high_rate.activation_energy
    elif rxn.reaction_type == 'pressure-dependent-Arrhenius':
        for p in rxn.rate.rates:
            p[1].pre_exponential_factor
            # p[1].temperature_exponent
            # p[1].activation_energy
    else:
        print(f"Reaction {i} has unhandled reaction type: {rxn.reaction_type}")
    

C:\Users\barki\AppData\Local\Temp\ipykernel_836\2247701750.py:7: UserWarning: NasaPoly2::validate: 
For species CH2OCH, discontinuity in cp/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

  gas = ct.Solution('C2H4_2021.yaml')
C:\Users\barki\AppData\Local\Temp\ipykernel_836\2247701750.py:7: UserWarning: NasaPoly2::validate: 
For species CH2OCH, discontinuity in h/RT detected at Tmid = 500
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

  gas = ct.Solution('C2H4_2021.yaml')
C:\Users\barki\AppData\Local\Temp\ipykernel_836\2247701750.py:7: UserWarning: NasaPoly2::validate: 
For species CH2OCH, discontinuity in s/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

  gas = ct.So

2 O + M <=> O2 + M
False


In [6]:
class Reaction:
    def __init__(self, id, f, nominal_arrhenius_params):
        self.id = id
        self.f = f
        self.nominal_arrhenius_params = nominal_arrhenius_params
        self.A_interval = None
    
    def calc_arrhenius_interval(self):
        # Here f is not temperature dependent
        # TODO: implment for f(T) 
        A_min = self.nominal_arrhenius_params[0] / np.pow(10, self.f)
        A_max = self.nominal_arrhenius_params[0] * np.pow(10, self.f)
        self.A_interval = (A_min, A_max)

In [7]:
# dataset of [sampled rates ,simulated IDT]


In [35]:
# load df from csv
if_df = pd.read_csv("impact_factors_ignition_delay.csv")

inactive_eqs = if_df.groupby("equation")["low_impact"].sum().eq(2)
if_df["active"] = ~if_df["equation"].map(inactive_eqs).fillna(False)



print(  if_df)

                       equation         f_range   id  f_value  \
0        C2H4 + O <=> CH3 + HCO  [0.135, 0.142]  278   0.1385   
1       C2H4 + O <=> CH2CO + H2  [0.135, 0.142]  279   0.1385   
2       C2H4 + O <=> CH2CHO + H  [0.135, 0.142]  280   0.1385   
3      C2H4 + OH <=> C2H3 + H2O  [0.301, 0.318]  271   0.3095   
4        C2H4 + OH <=> CH2CH2OH  [0.143, 0.149]  275   0.1460   
5   C2H4 + H (+M) <=> C2H5 (+M)    [0.22, 0.22]  269   0.2200   
6        C2H4 + H <=> C2H3 + H2  [0.847, 0.898]  270   0.8725   
7      C2H5 + O2 <=> C2H4 + HO2  [0.231, 0.241]  316   0.2360   
8          C2H5 + O2 <=> C2H5O2  [0.491, 0.511]  317   0.5010   
9        C2H4 + O <=> CH3 + HCO  [0.135, 0.142]  278   0.1385   
10      C2H4 + O <=> CH2CO + H2  [0.135, 0.142]  279   0.1385   
11      C2H4 + O <=> CH2CHO + H  [0.135, 0.142]  280   0.1385   
12     C2H4 + OH <=> C2H3 + H2O  [0.301, 0.318]  271   0.3095   
13       C2H4 + OH <=> CH2CH2OH  [0.143, 0.149]  275   0.1460   
14  C2H4 + H (+M) <=> C2H

In [8]:
def arrhenius_rate(A, n, Ea, T):
    R = 8.314  # J/(mol*K)
    k = A * (T ** n) * np.exp(-Ea / (R * T))
    return k